# Welch t test

### Utils

In [1]:
from scipy.stats import ttest_ind_from_stats

rows_with_numerical_results = [4, 6, 7, 8, 10]

def read_table_pre_review (exp):
    
    with open(exp+'_table_pre_review.txt', 'r') as f:
        rows = f.readlines()
        results = {}
        for i in rows_with_numerical_results:
            row = rows[i].split(' & ')
            layers = row[0]
            results[layers] = [get_mean_and_std(result) for result in row[1:]]
    return rows, results

def get_mean_and_std (result):
    '''
    expected format:
    1. '${90.0}_{\\pm 0.1}$'
    2. '$\\mathbf{99.2}_{\\pm 0.7}$'
    '''    
    tmp = result.split('}_{\\pm ')
    mean = tmp[0].split('{')[1]
    std = tmp[1].split('}')[0]
    return {'mean': float(mean), 'std': float(std), 'mean_str': mean, 'std_str': std}

# --------------------------------------------------------

couples = [(0, 1), (2, 3), (4, 5)]

def make_welch_t_test (results, alpha, nobs):
    
    for layers, result in results.items():
        for couple in couples:
            r1, r2 = result[couple[0]], result[couple[1]]
            _, pvalue = ttest_ind_from_stats(
                r1['mean'], r1['std'], nobs,
                r2['mean'], r2['std'], nobs,
                equal_var=False, # Welch
                alternative='two-sided'
            )
            r1['win'] = r1['mean'] >= r2['mean']
            r2['win'] = r2['mean'] >= r1['mean']
            r1['pvalue'] = pvalue
            r2['pvalue'] = pvalue
            r1['bold'] = r1['win'] or (pvalue > alpha and r2['mean'] < 100)
            r2['bold'] = r2['win'] or (pvalue > alpha and r1['mean'] < 100)
    return results

# --------------------------------------------------------

def save_table_post_review (exp, rows, results):
     
    for i, layers in zip(rows_with_numerical_results, results.keys()):
        rows[i] = layers + ' & ' + get_row_to_print(results[layers]) + ' \\\\\n'
    with open(exp+'_table_post_review.txt', 'w') as f:
        for row in rows: f.write(row)
            
def get_row_to_print (result):
    
    s = ''
    count = 0
    for r in result:
        if count != 0: s += ' & '
        s += my_latex_format(r)
        count += 1
    return s

def my_latex_format (r):

    if r['bold'] == True: return "$\\mathbf{" + r['mean_str'] + "}_{\\pm " + r['std_str'] + "}$"
    else: return "${" + r['mean_str'] + "}_{\\pm " + r['std_str'] + "}$"

***

### Main

In [2]:
for exp in ['2moon', 'nsy2moon', 'mnist']:

    rows, results = read_table_pre_review(exp)
    results = make_welch_t_test(results, alpha=0.05, nobs=5)
    save_table_post_review(exp, rows, results)